In [1]:
from networkx import DiGraph, erdos_renyi_graph, is_directed_acyclic_graph
import numpy as np
import pandas as pd

from causallearn.graph.Dag import Dag
from causallearn.graph.GraphNode import GraphNode
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.cit import chisq, fisherz, kci, d_separation
from causallearn.utils.DAG2PAG import dag2pag
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge

from causallearn.search.ScoreBased.GES import ges
from causallearn.search.ConstraintBased.PC import pc

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

/Users/fzhenjiang/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
file_path = "RWDataEstimation/PAM50/";
file_name = "BRCADataPM50.txt";
dataset = pd.read_csv(file_path+file_name,sep='\t');
display(dataset);
#print(mixed_sim_dataset.columns);

,CLEC3A,CPB1,SCGB2A2,SCGB1D2,TFF1,GSTM1,PIP,S100A7,MUCL1,CYP2B7P1,...,COL9A3,IRX1,AFP,LOC339535,PDZK1IP1,LOC84740,LOC440905,CDH1,ERBB2,PAM50Call_RNAseq
0,1.7015,6.6126,8.3479,7.7986,2.5742,0.0000,5.5251,0.0000,4.8089,1.8887,...,9.3336,8.3217,0.0000,7.2256,7.2343,4.2191,6.0709,14.5226,12.7126,1
1,4.0415,6.2547,10.8725,9.9367,13.1286,12.6025,7.3682,0.0000,10.6232,6.3334,...,5.9591,8.3767,1.2118,0.4104,8.7694,2.5703,6.1650,13.3517,13.1849,2
2,0.8928,6.5801,14.1237,10.6065,13.1892,10.5203,10.3385,0.0000,6.6495,8.7548,...,5.7351,7.9251,1.1923,2.2796,7.8969,3.2608,10.1397,13.9822,12.9618,2
3,0.0000,3.6913,17.1161,15.5172,9.8676,8.1795,7.9117,0.4449,9.6917,5.6983,...,6.5462,7.8543,3.5649,1.2896,6.4660,2.4181,5.8774,13.2457,14.1351,2
4,11.8279,6.9578,10.0564,5.5244,4.2665,0.9130,10.6559,0.0000,14.8774,8.0641,...,5.9106,5.2731,5.3992,4.8849,9.4650,3.7149,2.4368,13.3157,12.1305,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.0000,4.0715,3.1285,5.4674,0.0000,7.6400,2.9592,2.5462,0.7184,2.3772,...,6.1955,1.3860,0.4035,6.1438,11.5044,9.8210,5.9083,12.1229,11.6357,4
597,3.1862,11.6245,8.0968,4.8590,9.6830,0.0000,4.1047,0.7443,9.2764,10.6440,...,7.9311,5.7368,1.2328,2.6784,4.5618,1.8944,6.4104,14.7650,14.0283,2
598,15.5830,10.1516,15.6385,14.1262,12.4475,2.5281,12.7570,2.3597,16.5419,12.1057,...,4.5782,3.8431,0.9662,2.2674,11.6040,3.4868,4.8567,14.0134,12.7355,2
599,4.0102,19.2823,12.6806,10.2340,7.4761,7.0284,5.7131,1.8151,9.9208,9.0875,...,5.6524,6.4608,1.2865,4.9311,6.4067,3.3265,6.0473,15.2450,12.4689,2


In [9]:
PAM50GenesStr = "ACTR3B, ANLN, BAG1, BCL2, BIRC5, BLVRA, CCNB1, CCNE1, CDC20, CDC6, CDH3, CENPF, CEP55, CXXC5, EGFR, ERBB2, ESR1, EXO1, FGFR4, FOXA1, FOXC1, GPR160, GRB7, KIF2C, KRT14, KRT17, KRT5, MAPT, MDM2, MELK, MIA, MKI67, MLPH, MMP11, MYBL2, MYC, NAT1, NDC80, NUF2, ORC6L, PGR, PHGDH, PTTG1, RRM2, SFRP1, SLC39A6, TMEM45B, TYMS, UBE2C, UBE2T";
PAM50GenesList = list(PAM50GenesStr.split(", "));
PAM50GenesList

['ACTR3B',
 'ANLN',
 'BAG1',
 'BCL2',
 'BIRC5',
 'BLVRA',
 'CCNB1',
 'CCNE1',
 'CDC20',
 'CDC6',
 'CDH3',
 'CENPF',
 'CEP55',
 'CXXC5',
 'EGFR',
 'ERBB2',
 'ESR1',
 'EXO1',
 'FGFR4',
 'FOXA1',
 'FOXC1',
 'GPR160',
 'GRB7',
 'KIF2C',
 'KRT14',
 'KRT17',
 'KRT5',
 'MAPT',
 'MDM2',
 'MELK',
 'MIA',
 'MKI67',
 'MLPH',
 'MMP11',
 'MYBL2',
 'MYC',
 'NAT1',
 'NDC80',
 'NUF2',
 'ORC6L',
 'PGR',
 'PHGDH',
 'PTTG1',
 'RRM2',
 'SFRP1',
 'SLC39A6',
 'TMEM45B',
 'TYMS',
 'UBE2C',
 'UBE2T']

In [11]:
ges_list = [];

step_interval = 20;
step_index = step_interval;
while step_index <= 500:
    start_idx = step_index-step_interval;
    end_idx = step_index;
    
    if end_idx==500:
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:dataset.shape[1]];
    else:
        #The last patch
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:end_idx];
        temp_dataset["PAM50Call_RNAseq"] = dataset["PAM50Call_RNAseq"];
        
    temp_data_mat = temp_dataset.to_numpy();
    
    # Run GES with default parameters: score_func='local_score_BIC', maxP=None, parameters=None
    # Run GES and obtain the estimated graph (res_map is Dict object，which contains the updated steps, the result causal graph and the result score.)
    res_map = ges(temp_data_mat, score_func='local_score_BIC', maxP=None, parameters=None)  
    relation_list = res_map['update1'];

    name_index_mapping = {};
    index_name_mapping = {};

    colList = temp_dataset.columns.tolist();
    for index,name in enumerate(colList):
        name_index_mapping[name] = index;
        index_name_mapping[index] = name;

    for rel in relation_list:
        node1_name = index_name_mapping[rel[0]];
        node2_name = index_name_mapping[rel[1]];
        if node2_name == "PAM50Call_RNAseq" and node1_name in PAM50GenesList:
            print(node1_name+"->"+node2_name);
            ges_list.append({"Cause":node1_name,"Effect":node2_name});
        
    step_index=step_index+step_interval;

20
40
KRT14->PAM50Call_RNAseq
60
80
PGR->PAM50Call_RNAseq
100
KRT17->PAM50Call_RNAseq
120
ESR1->PAM50Call_RNAseq
140
160
180
200
220
240
260
280
NAT1->PAM50Call_RNAseq
300
320
FOXA1->PAM50Call_RNAseq
340
360
380
400
420
440
460
480
500
ERBB2->PAM50Call_RNAseq


In [12]:
fci_list = [];

step_interval = 20;
step_index = step_interval;
while step_index <= 500:
    start_idx = step_index-step_interval;
    end_idx = step_index;
    
    if end_idx==500:
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:dataset.shape[1]];
    else:
        #The last patch
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:end_idx];
        temp_dataset["PAM50Call_RNAseq"] = dataset["PAM50Call_RNAseq"];
        
    temp_data_mat = temp_dataset.to_numpy();

    name_index_mapping = {};
    index_name_mapping = {};

    colList = temp_dataset.columns.tolist();
    for index,name in enumerate(colList):
        name_index_mapping[name] = index;
        index_name_mapping[index] = name;

        
    G, edges = fci(temp_data_mat, fisherz, 0.05, verbose=False);
    
    for edge in edges:
        edge = str(edge);
        # print(edge);
        if " --> " in edge:
            edge_strs = edge.split(" --> ");
            node1_name = int(edge_strs[0].replace('X', ''))-1;
            node2_name = int(edge_strs[1].replace('X', ''))-1;

            node1_name = index_name_mapping[node1_name];
            node2_name = index_name_mapping[node2_name];

            if node2_name == "PAM50Call_RNAseq" and node1_name in PAM50GenesList:
                print(node1_name+"->"+node2_name);
                fci_list.append({"Cause":node1_name,"Effect":node2_name});
        
    step_index=step_index+step_interval;

20


  0%|          | 0/21 [00:00<?, ?it/s]

X7 --> X1
X1 --> X10
X16 --> X1
X2 --> X15
X18 --> X2
X10 --> X5
X5 --> X16
X16 --> X7
X16 --> X9
X9 --> X17
X10 --> X15
X10 --> X18
X12 --> X11
X17 --> X11
X16 --> X13
X17 --> X13
X14 --> X20
X15 --> X20
X21 --> X15
X18 --> X21
40


  0%|          | 0/21 [00:00<?, ?it/s]

X14 --> X1
X1 --> X19
X5 --> X11
X12 --> X6
X13 --> X6
X8 --> X7
X9 --> X15
X16 --> X10
X17 --> X10
X21 --> X10
X11 --> X16
X18 --> X13
X16 --> X14
X14 --> X19
X16 --> X17
X17 --> X21
60


...

X2 --> X5
X6 --> X3
X10 --> X3
X7 --> X4
X7 --> X5
X5 --> X17
X19 --> X5
X12 --> X6
X17 --> X6
X7 --> X14
X21 --> X9
X10 --> X18
X13 --> X17
480


  0%|          | 0/21 [00:00<?, ?it/s]

X21 --> X1
X6 --> X2
X18 --> X3
X20 --> X3
X9 --> X5
X16 --> X7
X7 --> X19
X21 --> X8
X9 --> X16
X11 --> X19
X21 --> X12
X16 --> X15
X19 --> X15
X19 --> X20
X21 --> X19
500


  0%|          | 0/23 [00:00<?, ?it/s]

X4 --> X1
X1 --> X6
X1 --> X10
X6 --> X2
X6 --> X3
X3 --> X23
X4 --> X14
X4 --> X15
X10 --> X5
X6 --> X7
X6 --> X14
X6 --> X19
X19 --> X11
X14 --> X18


In [13]:
pc_list = [];

step_interval = 20;
step_index = step_interval;
while step_index <= 500:
    start_idx = step_index-step_interval;
    end_idx = step_index;
    
    if end_idx==500:
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:dataset.shape[1]];
    else:
        #The last patch
        print(end_idx);
        temp_dataset = dataset.iloc[:, start_idx:end_idx];
        temp_dataset["PAM50Call_RNAseq"] = dataset["PAM50Call_RNAseq"];
        
    temp_data_mat = temp_dataset.to_numpy();

    name_index_mapping = {};
    index_name_mapping = {};

    colList = temp_dataset.columns.tolist();
    for index,name in enumerate(colList):
        name_index_mapping[name] = index;
        index_name_mapping[index] = name;

        
    # G, edges = fci(temp_data_mat, fisherz, 0.05, verbose=False);
    cg = pc(temp_data_mat, 0.05, fisherz, True, 0, 1);
    for edge in cg.G.get_graph_edges():
    # for edge in edges:
        edge = str(edge);
        # print(edge);
        if " --> " in edge:
            edge_strs = edge.split(" --> ");
            node1_name = int(edge_strs[0].replace('X', ''))-1;
            node2_name = int(edge_strs[1].replace('X', ''))-1;

            node1_name = index_name_mapping[node1_name];
            node2_name = index_name_mapping[node2_name];

            if node2_name == "PAM50Call_RNAseq" and node1_name in PAM50GenesList:
                print(node1_name+"->"+node2_name);
                pc_list.append({"Cause":node1_name,"Effect":node2_name});
                
        
    step_index=step_index+step_interval;

20


  0%|          | 0/21 [00:00<?, ?it/s]

40


  0%|          | 0/21 [00:00<?, ?it/s]

60


...

  0%|          | 0/21 [00:00<?, ?it/s]

480


  0%|          | 0/21 [00:00<?, ?it/s]

500


  0%|          | 0/23 [00:00<?, ?it/s]

In [14]:
ges_df = pd.DataFrame(ges_list);
display(ges_df);

fci_df = pd.DataFrame(fci_list);
display(fci_df);

pc_df = pd.DataFrame(pc_list);
display(pc_df);

#Save the results
ges_output_file_name = "ges_output_file.csv";
fci_output_file_name = "fci_output_file.csv";
pc_output_file_name = "pc_output_file.csv";

ges_df.to_csv(file_path+ges_output_file_name,index=None);
fci_df.to_csv(file_path+fci_output_file_name,index=None);
pc_df.to_csv(file_path+pc_output_file_name,index=None);

,Cause,Effect
0,KRT14,PAM50Call_RNAseq
1,PGR,PAM50Call_RNAseq
2,KRT17,PAM50Call_RNAseq
3,ESR1,PAM50Call_RNAseq
4,NAT1,PAM50Call_RNAseq
5,FOXA1,PAM50Call_RNAseq
6,ERBB2,PAM50Call_RNAseq


,Cause,Effect
0,PGR,PAM50Call_RNAseq
1,MAPT,PAM50Call_RNAseq


,Cause,Effect
0,NAT1,PAM50Call_RNAseq


In [ ]:
# temp_dataset = dataset.iloc[:, :10];

# temp_data_mat = temp_dataset.to_numpy();

# # Run PC with: stable=True, uc_rule=0 (uc_sepset), uc_priority=1 (orient bi-directed)
# cg = pc(temp_data_mat, 0.05, fisherz, True, 0, 1)
# for edge in cg.G.get_graph_edges():
#     edge = str(edge);
    
#     if " --> " in edge:
#         edge_strs = edge.split(" --> ");
#         node1_name = int(edge_strs[0].replace('X', ''))-1;
#         node2_name = int(edge_strs[1].replace('X', ''))-1;
#         print(edge);
        
#         node1_name = index_name_mapping[node1_name];
#         node2_name = index_name_mapping[node2_name];

#         if node2_name == "PAM50Call_RNAseq" and node1_name in PAM50GenesList:
#             print(node1_name+"->"+node2_name);

In [4]:
# 14---25___Incorrect filtering using independence tests..........................................
# 30---31___Incorrect filtering using independence tests..........................................
# 5---16___Incorrect filtering using independence tests..........................................
# 5---11___Incorrect filtering using independence tests..........................................
# 1---5___Incorrect filtering using independence tests..........................................
# 20---24___Incorrect filtering using independence tests..........................................
'''
{0: 'X30', 1: 'X9', 2: 'X16', 3: 'X14', 4: 'X7', 5: 'X17', 6: 'X28', 7: 'X5', 8: 'X21', 9: 'X6', 10: 'X2', 11: 'X12'
 , 12: 'X29', 13: 'X27', 14: 'X25', 15: 'X22', 16: 'X26', 17: 'X15', 18: 'X8', 19: 'X24', 20: 'X1', 21: 'X3', 22: 'X18'
 , 23: 'X4', 24: 'X19', 25: 'X20', 26: 'X10', 27: 'X23', 28: 'X11', 29: 'X13', 30: 'X31', 31: 'X39', 32: 'X32'
 , 33: 'X40', 34: 'X33', 35: 'X41', 36: 'X34', 37: 'X42', 38: 'X35', 39: 'X43', 40: 'X36', 41: 'X44', 42: 'X37', 43: 'X45'
 , 44: 'X38', 45: 'X46', 46: 'X47', 47: 'X48', 48: 'X49', 49: 'X50'}

{0: True, 1: True, 2: False, 3: False, 4: True, 5: False, 6: False, 7: False, 8: True, 9: False, 10: True, 11: True
 , 12: True, 13: True, 14: True, 15: True, 16: True, 17: False, 18: False, 19: False, 20: False, 21: True, 22: False
 , 23: False, 24: False, 25: False, 26: True, 27: False, 28: True, 29: True, 30: False, 31: False, 32: False
 , 33: False, 34: False, 35: False, 36: False, 37: False, 38: False, 39: False, 40: False, 41: False, 42: False
 , 43: False, 44: False, 45: False, 46: False, 47: False, 48: False, 49: False}
 
[(30, 31), (31, 46), (32, 33), (34, 35), (36, 37), (38, 39), (40, 41), (42, 43), (44, 45), (47, 49), (48, 49), (1, 12)
 , (1, 17), (1, 3), (1, 5), (1, 25), (17, 25), (17, 5), (3, 13), (3, 22), (5, 25), (5, 16), (5, 14), (5, 6), (25, 19)
 , (25, 14), (28, 25), (28, 15), (28, 12), (28, 2), (15, 13), (2, 27), (19, 0), (11, 19), (11, 2), (11, 5), (26, 16)
 , (26, 27), (26, 29), (27, 12), (21, 0), (21, 18), (21, 11), (21, 17), (21, 25), (21, 19), (18, 26), (18, 28), (18, 2)
 , (13, 0), (22, 8), (22, 16), (8, 15), (8, 14), (20, 23), (20, 9), (20, 17), (20, 24), (20, 16), (23, 9), (23, 15)
 , (9, 15), (10, 26), (10, 3), (10, 15), (10, 16), (7, 26), (7, 25), (7, 12), (4, 24), (4, 8)]
 
'''
#X20->X25, not a real causal relation
# x_label = 'X20';continuouse  25
# y_label = 'X25';discrete 14

# #30 31, X31->X39, a real causal relation  SOLVED
# x_label = 'X31';# continuous 30
# y_label = 'X39';# continuous 31

# #5->11, not a real causal relation
# x_label = 'X17';#  continuous 5
# y_label = 'X12';#  discrete 11

# #1->5, a real causal relation
# x_label = 'X9';# discrete 1
# y_label = 'X17';#  continuous 5

# #5->16, a real causal relation
# x_label = 'X17';#  continuous 5
# y_label = 'X26';#  discrete 16

# #20->24, a real causal relation
# x_label = 'X1';# continuous 20
# y_label = 'X19';# continuous 24

# sns.regplot(data=mixed_nonlinear_sim_data, x=x_label, y=y_label, robust=True);

"\n{0: 'X30', 1: 'X9', 2: 'X16', 3: 'X14', 4: 'X7', 5: 'X17', 6: 'X28', 7: 'X5', 8: 'X21', 9: 'X6', 10: 'X2', 11: 'X12'\n , 12: 'X29', 13: 'X27', 14: 'X25', 15: 'X22', 16: 'X26', 17: 'X15', 18: 'X8', 19: 'X24', 20: 'X1', 21: 'X3', 22: 'X18'\n , 23: 'X4', 24: 'X19', 25: 'X20', 26: 'X10', 27: 'X23', 28: 'X11', 29: 'X13', 30: 'X31', 31: 'X39', 32: 'X32'\n , 33: 'X40', 34: 'X33', 35: 'X41', 36: 'X34', 37: 'X42', 38: 'X35', 39: 'X43', 40: 'X36', 41: 'X44', 42: 'X37', 43: 'X45'\n , 44: 'X38', 45: 'X46', 46: 'X47', 47: 'X48', 48: 'X49', 49: 'X50'}\n\n{0: True, 1: True, 2: False, 3: False, 4: True, 5: False, 6: False, 7: False, 8: True, 9: False, 10: True, 11: True\n , 12: True, 13: True, 14: True, 15: True, 16: True, 17: False, 18: False, 19: False, 20: False, 21: True, 22: False\n , 23: False, 24: False, 25: False, 26: True, 27: False, 28: True, 29: True, 30: False, 31: False, 32: False\n , 33: False, 34: False, 35: False, 36: False, 37: False, 38: False, 39: False, 40: False, 41: False, 42: